<a href="https://colab.research.google.com/github/will-cotton4/DS-Unit-2-Sprint-3-Classification-Validation/blob/master/DS_Unit_2_Sprint_Challenge_3_Classification_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 _Lambda School Data Science Unit 2_
 
 # Classification & Validation Sprint Challenge

Follow the instructions for each numbered part to earn a score of 2. See the bottom of the notebook for a list of ways you can earn a score of 3.

#### For this Sprint Challenge, you'll predict whether a person's income exceeds $50k/yr, based on census data.

You can read more about the Adult Census Income dataset at the UCI Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/adult

#### Run this cell to load the data:

In [0]:
import pandas as pd

columns = ['age', 
           'workclass', 
           'fnlwgt', 
           'education', 
           'education-num', 
           'marital-status', 
           'occupation', 
           'relationship', 
           'race', 
           'sex', 
           'capital-gain', 
           'capital-loss', 
           'hours-per-week', 
           'native-country', 
           'income']

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', 
                 header=None, names=columns)

df['income'] = df['income'].str.strip()

## Part 1 — Begin with baselines

Split the data into an **X matrix** (all the features) and **y vector** (the target).

(You _don't_ need to split the data into train and test sets here. You'll be asked to do that at the _end_ of Part 1.)

In [0]:
X = df.drop(columns='income')
# Make vector easier to deal with:
y = df['income'] == '>50K'

What **accuracy score** would you get here with a **"majority class baseline"?** 
 
(You can answer this question either with a scikit-learn function or with a pandas function.)

In [13]:
#Check the majority class (most common value):
y.value_counts()

False    24720
True      7841
Name: income, dtype: int64

In [0]:
#Predict:
majority_class = y.mode()[0]
y_pred = [majority_class] * len(y)

In [15]:
from sklearn.metrics import accuracy_score

accuracy_score(y, y_pred)

0.7591904425539756

What **ROC AUC score** would you get here with a **majority class baseline?**

(You can answer this question either with a scikit-learn function or with no code, just your understanding of ROC AUC.)

In [17]:
# My guess: it should be 0.5, since ROC-AUC measures things relative to chance,
# with 0.5 indicating that you're not gaining anything relative to chance.

from sklearn.metrics import roc_auc_score

roc_auc_score(y, y_pred)

0.5

**Nice.**

In this Sprint Challenge, you will use **"Cross-Validation with Independent Test Set"** for your model validaton method.

First, **split the data into `X_train, X_test, y_train, y_test`**. You can include 80% of the data in the train set, and hold out 20% for the test set.

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.2, random_state = 42, stratify = y)

## Part 2 — Modeling with Logistic Regression!

- You may do exploratory data analysis and visualization, but it is not required.
- You may **use all the features, or select any features** of your choice, as long as you select at least one numeric feature and one categorical feature.
- **Scale your numeric features**, using any scikit-learn [Scaler](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing) of your choice.
- **Encode your categorical features**. You may use any encoding (One-Hot, Ordinal, etc) and any library (category_encoders, scikit-learn, pandas, etc) of your choice.
- You may choose to use a pipeline, but it is not required.
- Use a **Logistic Regression** model.
- Use scikit-learn's [**cross_val_score**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) function. For [scoring](https://scikit-learn.org/stable/modules/model_evaluation.html#the-scoring-parameter-defining-model-evaluation-rules), use **accuracy**.
- **Print your model's cross-validation accuracy score.**

In [21]:
!pip install category_encoders

     |████████████████████████████████| 92kB 3.5MB/s 


In [26]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from category_encoders import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

pipeline = make_pipeline(
    OneHotEncoder(),
    StandardScaler(),
    LogisticRegression(solver='lbfgs', max_iter=1000)
)

scores = cross_val_score(pipeline, X_train, y_train, scoring='accuracy', cv=10, n_jobs=-1, verbose=10)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   10.5s finished


In [27]:
print(scores.mean())

0.8507752447956076


**Let's see how we're doing with the ROC-AUC score:**

In [28]:
scores = cross_val_score(pipeline, X_train, y_train, scoring='roc_auc', cv=10, n_jobs=-1, verbose=10)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   11.3s finished


In [29]:
print(scores.mean())

0.9057967486367534


**_Nice._**

## Part 3 — Modeling with Tree Ensembles!

Part 3 is the same as Part 2, except this time, use a **Random Forest** or **Gradient Boosting** classifier. You may use scikit-learn, xgboost, or any other library. Then, print your model's cross-validation accuracy score.

In [39]:
from sklearn.ensemble import RandomForestClassifier

forest_pipeline = make_pipeline(
    OneHotEncoder(),
    StandardScaler(),
    RandomForestClassifier(max_depth=3, n_estimators=100, n_jobs=-1, random_state=42) 
)

scores = cross_val_score(forest_pipeline, X_train, y_train,
                         scoring='accuracy', cv=10, n_jobs=-1, verbose=10)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   11.0s finished


In [40]:
print(scores.mean())

0.7905397831627359


**And ROC-AUC:**

In [41]:
scores = cross_val_score(forest_pipeline, X_train, y_train,
                         scoring='roc_auc', cv=10, n_jobs=-1, verbose=10)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   10.8s finished


In [42]:
print(scores.mean())

0.8861276804831839


**Now with gradient boosting:**

In [43]:
from xgboost import XGBClassifier
gb_pipeline = make_pipeline(
    OneHotEncoder(),
    StandardScaler(),
    XGBClassifier(max_depth=3, n_estimators=100, n_jobs=-1, random_state=42) 
)

scores = cross_val_score(gb_pipeline, X_train, y_train,
                         scoring='accuracy', cv=10, n_jobs=-1, verbose=10)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   11.2s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   22.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   55.3s finished


In [44]:
print(scores.mean())

0.8636366341585976


In [45]:
scores = cross_val_score(gb_pipeline, X_train, y_train,
                         scoring='roc_auc', cv=10, n_jobs=-1, verbose=10)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   11.0s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   22.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   55.4s finished


In [46]:
print(scores.mean())

0.9199943720555522


**NICE**

## Part 4 — Calculate classification metrics from a confusion matrix

Suppose this is the confusion matrix for your binary classification model:

<table>
  <tr>
    <td colspan="2" rowspan="2"></td>
    <td colspan="2">Predicted</td>
  </tr>
  <tr>
    <td>Negative</td>
    <td>Positive</td>
  </tr>
  <tr>
    <td rowspan="2">Actual</td>
    <td>Negative</td>
    <td style="border: solid">85</td>
    <td style="border: solid">58</td>
  </tr>
  <tr>
    <td>Positive</td>
    <td style="border: solid">8</td>
    <td style="border: solid"> 36</td>
  </tr>
</table>

In [0]:
tn = 85
tp = 36

fp = 58
fn = 8

Calculate accuracy

In [49]:
accuracy = (tp + tn)/ (tp+tn+fp+fn)
print(accuracy)

0.6470588235294118


Calculate precision

In [50]:
precision = tp / (tp+fp)
print(precision)

0.3829787234042553


Calculate recall

In [51]:
recall = tp / (tp + fn)
print(recall)

0.8181818181818182


## BONUS — How you can earn a score of 3

### Part 1
Do feature engineering, to try improving your cross-validation score.

### Part 2
Experiment with feature selection, preprocessing, categorical encoding, and hyperparameter optimization, to try improving your cross-validation score.

### Part 3
Which model had the best cross-validation score? Refit this model on the train set and do a final evaluation on the held out test set — what is the test score? 

### Part 4
Calculate F1 score and False Positive Rate. 